In [1]:
"""
Γεμίζω τον πίνακα με τις μετρήσεις
του chartevet 8ώρου.

4.177.458 εγγραφές περιλαμβάνονται
στον πίνακα με τις μέσες τιμές οχταώρου.

6.646.751 εγγραφές είναι κενές
"""

'\nΓεμίζω τον πίνακα με τις μετρήσεις\nτου chartevet 8ώρου.\n\n4.177.458 εγγραφές περιλαμβάνονται\nστον πίνακα με τις μέσες τιμές οχταώρου.\n\n6.646.751 εγγραφές είναι κενές\n'

In [2]:
import pandas as pd

In [3]:
df_chart_tbl = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o9_empty_chart_tbl.csv', index_col=False)
df_data_chartevents = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\icu\d_items.csv.gz')
df_grouped_cerebral_infarction = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o9_cerebral_infarction_chartevents_grouped_8hours_date_time.csv', index_col=False)

In [4]:
# Φέρνω τις τελευταίες τέσσερις στήλες πρώτες
cols = list(df_chart_tbl.columns)
cols = [cols[-1]] + cols[:-1]
df_chart_tbl = df_chart_tbl[cols]

cols = list(df_chart_tbl.columns)
cols = [cols[-1]] + cols[:-1]
df_chart_tbl = df_chart_tbl[cols]

cols = list(df_chart_tbl.columns)
cols = [cols[-1]] + cols[:-1]
df_chart_tbl = df_chart_tbl[cols]

cols = list(df_chart_tbl.columns)
cols = [cols[-1]] + cols[:-1]
df_chart_tbl = df_chart_tbl[cols]

In [5]:
# Ελέγχω αν υπάρχει NaN στην στήλη με τις μέσες τιμές, το αποτέλεσμα ισούται με μηδέν.
df_grouped_cerebral_infarction['Mean_Chart'].isna().sum()

0

In [6]:
df_grouped_cerebral_infarction = pd.merge(df_grouped_cerebral_infarction, df_data_chartevents, on=['itemid'])

In [7]:
# Δημιουργώ εκ νέου την στήλη concat χωρίς κενά ανάμεσα στα στοιχεία.
# Θα χρησιμοποιήσω αυτή τη στήλη σαν index γιατί τα κενά μπορεί να δημιουργήσουν πρόβλημα.
df_chart_tbl["concat"] = (df_chart_tbl["subject_id"].astype(str)
                          + df_chart_tbl["hadm_id"].astype(str)
                          + df_chart_tbl["date"].astype(str)
                          + df_chart_tbl["time"].astype(str))

# Δημιουργώ και την στήλη department έτσι ώστε να συμπληρωθεί το τμήμα
# από το οποίο έγινε η καταγραφή. Δεν έχει λειτουργική σημασία στο
# σημείο που είμαστε αλλά θα χρειαστεί ΄στον ενοποιημένο πίνακα.
df_chart_tbl.insert(2, 'Department', '')

In [8]:
# Αφαιρώ τους χαρακτήρες - και : από της τιμές της concat για να αποφύγω πιθανά προβλήματα
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace('-','')
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace(':','')

In [9]:
# Ορίζω την στήλη concat ως index των γραμμών
df_chart_tbl = df_chart_tbl.set_index('concat')

In [10]:
# Δημιουργώ την στήλη concat και στον πίνακα που περιλαμβάνει τις μετρήσεις για να είναι το κοινό σημείο ανάμεσα στους δυο πίνακες
df_grouped_cerebral_infarction["concat"] = (df_grouped_cerebral_infarction["subject_id"].astype(str)
                                            + df_grouped_cerebral_infarction["hadm_id"].astype(str)
                                            + df_grouped_cerebral_infarction["date"].astype(str)
                                            + df_grouped_cerebral_infarction["time"].astype(str))

In [11]:
#Δημιουργώ και την στήλη Department
df_grouped_cerebral_infarction.insert(0, 'Department', 'icu')

In [12]:
# Αφαιρώ τους χαρακτήρες - και : από της τιμές της concat για να αποφύγω πιθανά προβλήματα
df_grouped_cerebral_infarction['concat'] = df_grouped_cerebral_infarction['concat'].str.replace('-','')
df_grouped_cerebral_infarction['concat'] = df_grouped_cerebral_infarction['concat'].str.replace(':','')

In [13]:
# Μεταφέρω την τελευταία στήλη πρώτη
cols = list(df_grouped_cerebral_infarction.columns)
cols = [cols[-1]] + cols[:-1]
df_grouped_cerebral_infarction = df_grouped_cerebral_infarction[cols]

In [14]:
for row in df_grouped_cerebral_infarction.itertuples():
    con = row.concat
    item = row.itemid
    value = row.Mean_Chart
    df_chart_tbl.at['{}'.format(con), 'Department'] = row.Department
    #print('{}'.format(con), '{}'.format(item), '{}'.format(value))
    df_chart_tbl.at['{}'.format(con), '{}'.format(item)] = value

In [15]:
df_chart_tbl

,subject_id,hadm_id,Department,date,time,220045,220046,220047,220050,220051,...,228396,228869,228870,228872,228873,228874,228875,228876,229319,229355
concat,,,,,,,,,,,,,,,,,,,,,
100047332741187621741204080000,10004733,27411876,icu,2174-12-04,08:00:00,93.200000,130.0,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741204160000,10004733,27411876,icu,2174-12-04,16:00:00,97.250000,130.0,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205000000,10004733,27411876,icu,2174-12-05,00:00:00,100.333333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205080000,10004733,27411876,icu,2174-12-05,08:00:00,90.687500,120.0,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205160000,10004733,27411876,icu,2174-12-05,16:00:00,92.111111,120.0,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105468852521358721740929080000,10546885,25213587,icu,2174-09-29,08:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111148582403775721280812080000,11114858,24037757,icu,2128-08-12,08:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196236402939820421680401160000,19623640,29398204,icu,2168-04-01,16:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
"""
Ο πίνακας μου απαρτίζεται από 707 στήλες και 65189 γραμμές οπότε μου δημιουργούν 46.088.623 κελιά.

Οι τιμές μου ανέρχονται στα 4.177.458 οπότε με μια αφαίρεση προκείπτουν 41.911.165 NaN κελιά

Όταν οι κεφαλίδες ήταν οι κωδικοί των παρατηρήσεων οι καταγραφές ήταν ολοκληρωμένες,
όταν τις άλλαξε με τις ονομασίες 12 καταγραφές δεν έχουν περαστεί.
"""

# Η εντολή επαληθεύει το παραπάνω αποτέλεσμα.
df_chart_tbl.isna().sum().sum()

41911165

In [17]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τους κωδικούς των παρατηρήσεων
df_chart_tbl.to_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o9_fill_chart_itemid_header_tbl.csv', index=False)